In [1]:
import os

In [2]:
%pwd

'e:\\Fruit Freshness Classification\\Fruit-Freshness-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [5]:
from src.cnnClassifier.constants import *
# from src.cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAM_FILE_PATH):

                 self.config=read_yaml(config_filepath)
                 self.params=read_yaml(params_filepath)

                 create_directories([self.config.artifacts_root])

                 
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
               root_dir=config.root_dir,
               source_URL=config.source_URL,
               local_data_file=config.local_data_file,
               unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}") 

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
            


In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-20 17:50:54,887:INFO:common:yaml <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2026-01-20 17:50:54,890:INFO:common:yaml <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2026-01-20 17:50:54,891:INFO:common:created directory at artifacts]
[2026-01-20 17:50:54,892:INFO:common:created directory at artifacts/data_ingestion]
[2026-01-20 17:51:02,474:INFO:2914256545:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23504253
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "258a6ad2fd2515ce904992ee825ac3e32e9876c422a1b3ce3ca519a106c1b11c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 080D:1D41AA:169EBB:350048:696F732A
Accept-Ranges: bytes
Date: Tue, 20 Ja